# <center>Word2Vec Tutorial</center>
<center>Kelas Pengolahan Bahasa Manusia 2017/2018</center>
<center>Lintang Adyuta Sutawika</center>

In [ ]:
from input_data import InputData
import numpy
from model import SkipGramModel
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import sys
import os
import torch
from nltk.tokenize import RegexpTokenizer

In [ ]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)

class Corpus(object):
    def __init__(self, path='data'):
        self.dictionary = Dictionary()
        self.dataset = self.tokenize(os.path.join(path, 'ptb.train.txt'))

    def tokenize(self, path):
        self.tokenizer = RegexpTokenizer(r'\S+')
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = list(self.tokenizer.tokenize(line))
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r') as f:
            ids = torch.LongTensor(tokens)
            token = 0
            for line in f:
                # words = line.split() + ['<eos>']
                words = list(self.tokenizer.tokenize(line))
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1

        return ids

In [ ]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    tensor = torch.LongTensor(idxs)
    return Variable(tensor)

def get_index_of_max(input):
    index = 0
    for i in range(1, len(input)):
        if input[i] > input[index]:
            index = i 
    return index

def get_max_prob_result(input, ix_to_word):
    return ix_to_word[get_index_of_max(input)]

def get_similarity(w1, w2, model):
    e1 = model.get_word_emdedding(w1)
    e2 = model.get_word_emdedding(w2)
    return (e1.dot(e2) / (torch.norm(e1) * torch.norm(e2))).data.numpy()[0]

In [ ]:
#Continous Bag-of-Words Implementation
class cbow(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim):
        super(cbow, self).__init__()
        
        self.embeddings = nn.Embedding(vocabulary_size, embedding_dim)
        self.linear_output = nn.Linear(embedding_dim, vocabulary_size)
        
    def forward(self, imputs):
        projection = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear_output(projection)
        return nn.LogSoftmax(out)
        

In [ ]:
corpus = Corpus()
model = cbow(corpus.dictionary.len)

loss_function = nn.NLLLoss() 

for epoch in range(num_epoch):
    total_loss = 0
    for context, target in dataset:
        input_context = Variable(torch.LongTensor(corpus.dictionary.word2idx[word] for word in context))
        output_target = Variable(torch.LongTensor(corpus.dictionary.word2idx[target]))
        
        loss = loss_function(log_prob, target_var)
        loss.backward()
        optimizer.step()
        total_loss += loss.data

https://arxiv.org/pdf/1301.3781.pdf

<img src="Word_pair_relationships.png">

<img src="training_data.png">

<img src="CBOW_Skip-gram.png">

# Referensi
1) http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
